In [31]:
## Deliverable 2. Create a Customer Travel Destinations Map.
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/random_cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Quang Ngai,VN,2022-03-10 16:58:35,15.1167,108.8000,66.81,91,72,5.30,broken clouds
1,1,Manjeshwar,IN,2022-03-10 16:58:35,12.7194,74.8861,83.35,82,3,6.38,clear sky
2,2,Mindelo,CV,2022-03-10 16:58:35,16.8901,-24.9804,74.75,53,40,31.07,scattered clouds
3,3,Santa Maria,BR,2022-03-10 16:58:36,-29.6842,-53.8069,76.30,83,75,4.61,moderate rain
4,4,Vardo,NO,2022-03-10 16:58:36,70.3705,31.1107,30.18,80,98,36.82,overcast clouds


In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
vacation_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Manjeshwar,IN,2022-03-10 16:58:35,12.7194,74.8861,83.35,82,3,6.38,clear sky
3,3,Santa Maria,BR,2022-03-10 16:58:36,-29.6842,-53.8069,76.30,83,75,4.61,moderate rain
9,9,Bubaque,GW,2022-03-10 16:58:38,11.2833,-15.8333,76.17,71,100,17.09,overcast clouds
15,15,Bathsheba,BB,2022-03-10 16:58:40,13.2167,-59.5167,84.81,70,40,23.02,scattered clouds
18,18,Hithadhoo,MV,2022-03-10 16:56:15,-0.6000,73.0833,82.76,61,9,5.12,clear sky
...,...,...,...,...,...,...,...,...,...,...,...
664,664,Luba,GQ,2022-03-10 17:17:43,3.4568,8.5547,80.98,83,100,7.96,overcast clouds
671,671,Ubaitaba,BR,2022-03-10 17:17:47,-14.3125,-39.3233,89.91,48,26,6.53,scattered clouds
681,681,Salalah,OM,2022-03-10 17:14:43,17.0151,54.0924,75.07,55,1,0.54,clear sky
682,682,Nortelandia,BR,2022-03-10 17:17:51,-14.4547,-56.8028,82.42,69,98,6.20,overcast clouds


In [34]:
# 4a. Determine if there are any empty rows.


vacation_df.loc[(vacation_df["Country"].isnull())]

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
316,316,Khorixas,NaN,2022-03-10 17:07:03,-20.3667,14.9667,89.96,29,59,4.65,broken clouds


In [35]:
vacation_df = vacation_df.dropna()
vacation_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [36]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Manjeshwar,IN,83.35,clear sky,12.7194,74.8861,
3,Santa Maria,BR,76.30,moderate rain,-29.6842,-53.8069,
9,Bubaque,GW,76.17,overcast clouds,11.2833,-15.8333,
15,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167,
18,Hithadhoo,MV,82.76,clear sky,-0.6000,73.0833,
19,Kampene,CD,76.66,overcast clouds,-3.6000,26.6667,
20,Santo Domingo,DO,85.68,scattered clouds,18.5001,-69.9886,
21,Port Elizabeth,ZA,75.49,clear sky,-33.9180,25.5701,
24,Cidreira,BR,79.95,overcast clouds,-30.1811,-50.2056,
27,Rikitea,PF,78.46,broken clouds,-23.1203,-134.9692,


In [37]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [38]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found .... skipping")
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
Hotel not found .... skipping
-----------------------------
Data Retrieval Complete      
-----------------------------


In [40]:
test = hotel_df

In [51]:
        
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace("", np.nan).dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Manjeshwar,IN,83.35,clear sky,12.7194,74.8861,Mangalore Beach House - EWA Ikigai
3,Santa Maria,BR,76.30,moderate rain,-29.6842,-53.8069,Hotel Morotin
9,Bubaque,GW,76.17,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
15,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167,Atlantis Hotel
18,Hithadhoo,MV,82.76,clear sky,-0.6000,73.0833,Scoop Guest House
...,...,...,...,...,...,...,...
659,Iracoubo,GF,80.53,broken clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
671,Ubaitaba,BR,89.91,scattered clouds,-14.3125,-39.3233,Pousada Paraíso
681,Salalah,OM,75.07,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
682,Nortelandia,BR,82.42,overcast clouds,-14.4547,-56.8028,Hotel Karysmã


In [50]:
# 8a. Create the output File (CSV)
output_file = "hotel_info.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt> Hotel Name </dt><dd> {Hotel Name} </dd>
<dt> City Name </dt><dd> {City} </dd>
<dt> Country </dt><dd> {Country} </dd>
<dt> Weather description </dt><dd> {Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [62]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
# 11a. Add a marker layer for each city to the map.

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

fig

# 11b. Display the figure

Figure(layout=FigureLayout(height='420px'))